In [11]:
import numpy as np
import scipy.linalg as la
import mne.filter as bandpass
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import geomstats as gs
import geomstats.geometry.spd_matrices as spd
from geomstats.learning.frechet_mean import FrechetMean
from copy import deepcopy
from scipy.io import loadmat  
import pickle

In [15]:
class RCSP_KNN:
    def __init__(self, subjects, epochs, nchannels):
        self.subjects=subjects
        self.epochs=epochs
        self.nchannels = nchannels
      
    def bandpass(self, X, fs, fl, fh):
        X = np.swapaxes(X, 1, 2)
        X = bandpass.filter_data(X, fs, fl, fh, verbose=False)
        X = np.swapaxes(X, 1, 2)
        return X
        
    def calculateCovariances(self, X):
        covlist = np.empty((X.shape[0], X.shape[2], X.shape[2]))
        for i in range(len(X)):
            epoch = X[i]
            covlist[i] = np.dot(epoch.T, epoch)
        return covlist
    
    def estimateMeans(self, classSpecificCOV, metric):
        if metric == "classic":
            class0_avg = sum(classSpecificCOV[0])/len(classSpecificCOV[0])
            class1_avg = sum(classSpecificCOV[1])/len(classSpecificCOV[1])
            return [class0_avg, class1_avg]
        elif metric=="AIRM":
            estimator = FrechetMean(spd.SPDMetricAffine(n=self.nchannels), max_iter=64)
        elif metric=="LEM":
            estimator = FrechetMean(spd.SPDMetricLogEuclidean(n=self.nchannels), max_iter=64)
        elif metric=="BW":
            estimator = FrechetMean(spd.SPDMetricBuresWasserstein(n=self.nchannels), max_iter=64) #doesn't work yet
        else:
            raise Exception("Not implemented metric")
            
        means = []
        
        for COV in classSpecificCOV:
            estimator.fit(COV)
            mean = estimator.estimate_
            means.append(mean)
        return means
    
    
    def separate_classes(self, X, Y):
        classSpecificCOV = []
        for i in range(2): 
            ind = np.where(Y==i)[0]
            classCOV = X[ind]
            classSpecificCOV.append(classCOV)
        return classSpecificCOV
    
    def unite_classes(self, classSpecificCOV):
        X = np.concatenate((classSpecificCOV[0], classSpecificCOV[1]))
        Y = np.concatenate((np.zeros((len(classSpecificCOV[0]))), np.ones((len(classSpecificCOV[1])))))
        shuffle = np.random.permutation(len(X))
        
        X = X[shuffle]
        Y = Y[shuffle]
        
        return X, Y
    
    def CSP(self, classSpecificCOV, metric, n):
        class0_avg, class1_avg = self.estimateMeans(classSpecificCOV, metric)
        _,V = la.eigh(class0_avg, class0_avg+class1_avg)
        V = np.concatenate((V[:, :n], V[:, -n:]), axis=1)
        return V
    
    def applyCSP(self, dk, V):
        a = np.dot(np.dot(V.T, dk), V) 
        f = np.log(np.diagonal(a)/np.trace(a)) 
        return f

    
    def collectSubjects(self, session, fs=1000, fl=8, fh=30):
        channels = [7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 39, 40] #standard 10/20 cap mapping, electrodes responsible for MI
        
        SubjectsCOV = []
        SubjectsY = []
        
        for i in range(1, self.subjects+1):
                
            filename = 'datasets/54subjects/'+ session + '/subj{}_EEG_MI.mat'.format(i)
            data = loadmat(filename)
            
            d_train = data['EEG_MI_train']
            d_test = data['EEG_MI_test']
            
            X_train = d_train['smt'][0][0][1000:3500, :, channels]
            X_train_filtered = self.bandpass(np.swapaxes(X_train, 0, 1), fs, fl, fh)
            Y_train = d_train['y_dec'][0][0][0] - 1
                
            X_test = d_test['smt'][0][0][1000:3500, :, channels]
            X_test_filtered = self.bandpass(np.swapaxes(X_test, 0, 1), fs, fl, fh)
            Y_test = d_test['y_dec'][0][0][0] - 1
            
            X = np.concatenate((X_train_filtered, X_test_filtered))
            covX = self.calculateCovariances(X)
            Y = np.concatenate((Y_train, Y_test))
            
            SubjectsCOV.append(covX)
            SubjectsY.append(Y)
                    
        return SubjectsCOV, SubjectsY
   
    def distance(self, A, B, metric):
        if metric == "classic":
            return np.mean(np.abs(np.subtract(A, B)))
        elif metric == "AIRM":
            mt = spd.SPDMetricAffine(n=self.nchannels)
        elif metric == "LEM":
            mt = spd.SPDMetricLogEuclidean(n=self.nchannels)
        elif metric == "BW":
            mt = spd.SPDMetricBuresWasserstein(n=self.nchannels)
        else: 
            raise Exception("Not implemented metric")
        
        return mt.dist(A, B)
    
    def generateTrainTest(self, subject, classSpecificCOVs, SubjectsMeans, metric, samples, k, use_orig_subject_data):
        classSpecificCOVs = deepcopy(classSpecificCOVs)
        SubjectsMeans = deepcopy(SubjectsMeans)
        
        subjectSepCOVs = classSpecificCOVs.pop(subject) 
        SubjectsMeans.pop(subject)
        
        targetCOVs = [subjectSepCOVs[0][:samples], subjectSepCOVs[1][:samples]]
        
        testCOVs = [subjectSepCOVs[0][samples:], subjectSepCOVs[1][samples:]]
        
        targetCOVMeans = self.estimateMeans(targetCOVs, metric)
        
        class0dist = []
        class1dist = []
        
        for i in range(self.subjects-1):
            class0dist.append(self.distance(targetCOVMeans[0], SubjectsMeans[i][0], metric))
            class1dist.append(self.distance(targetCOVMeans[1], SubjectsMeans[i][1], metric))
        
        
        class0distSortIndex = sorted(range(len(class0dist)), key=lambda x: class0dist[x])[:k]
        class1distSortIndex = sorted(range(len(class1dist)), key=lambda x: class1dist[x])[:k]
        
        class0 = None
        class1 = None
        
        for i in range(self.subjects-1):
            if i in class0distSortIndex:
                if class0 is None:
                    class0 = classSpecificCOVs[i][0]
                else:
                    class0 = np.concatenate((class0, classSpecificCOVs[i][0]))
                    
            if i in class1distSortIndex:
                if class1 is None:
                    class1 = classSpecificCOVs[i][1]
                else:
                    class1 = np.concatenate((class1, classSpecificCOVs[i][1]))
                            
      
        if use_orig_subject_data:
            class0 = np.concatenate((class0, targetCOVs[0])) 
            class1 = np.concatenate((class1, targetCOVs[1]))
        
        trainCOVs = [class0, class1]
        
        return trainCOVs, testCOVs


    def mainloop(self, session, metric, n=3, samples=5, k=5, fs=1000, fl=8, fh=30, use_orig_subject_data=False):
        
        #SubjectsCOV, SubjectsY = self.collectSubjects(session, fs, fl, fh)
        SubjectsCOV, SubjectsY = pickle.load(open('datasets/54COVSess01.pickle', 'rb'))
        
        SubjectsMeans = []
        classSpecificCOVs = []
        for i in range(self.subjects):
            SeparatedCOV = self.separate_classes(SubjectsCOV[i], SubjectsY[i])
            classSpecificCOVs.append(SeparatedCOV)
            SubjectsMeans.append(self.estimateMeans(SeparatedCOV, metric))    
        
        results = []
        for ii in range(self.subjects):
            trainCOVs, testCOVs = self.generateTrainTest(ii, classSpecificCOVs, SubjectsMeans, metric, samples, k, use_orig_subject_data)
            
            V = self.CSP(trainCOVs, metric, n)
            clf = LDA()
            
            trainX, trainY = self.unite_classes(trainCOVs)
            testX, testY = self.unite_classes(testCOVs)
            
            train_features = np.empty((len(trainX), 2*n))
            
            for j in range(len(trainX)):
                epoch = trainX[j]
                train_features[j] = self.applyCSP(epoch, V)
                
            test_features = np.empty((len(testX), 2*n))
            
            for j in range(len(testX)):
                epoch = testX[j]
                test_features[j] = self.applyCSP(epoch, V)
                
            clf.fit(train_features, trainY)
            result = clf.score(test_features, testY)
            results.append(result)
            print("Subject " + str(ii+1) + ", Sub-Indep. Accuracy = " + str(result))
            
        av = sum(results)/len(results)
        print("Average: " + str(av))
    
        return results 

In [16]:
csp = RCSP_KNN(54, 200, 20)

In [18]:
metric = "AIRM"
sess = "sess01"

for n in range(2,6):
    print("\nN = " + str(n))
    for k in range(3, 10, 2):
        print("\nK = " + str(k) + "\n")
        res = csp.mainloop(sess, metric, n=n, samples=10, k=k)


N = 2

K = 3

Subject 1, Sub-Indep. Accuracy = 0.6833333333333333
Subject 2, Sub-Indep. Accuracy = 0.55
Subject 3, Sub-Indep. Accuracy = 0.7222222222222222
Subject 4, Sub-Indep. Accuracy = 0.5833333333333334
Subject 5, Sub-Indep. Accuracy = 0.7111111111111111
Subject 6, Sub-Indep. Accuracy = 0.5388888888888889
Subject 7, Sub-Indep. Accuracy = 0.49444444444444446
Subject 8, Sub-Indep. Accuracy = 0.5
Subject 9, Sub-Indep. Accuracy = 0.5722222222222222
Subject 10, Sub-Indep. Accuracy = 0.5888888888888889
Subject 11, Sub-Indep. Accuracy = 0.5166666666666667
Subject 12, Sub-Indep. Accuracy = 0.5611111111111111
Subject 13, Sub-Indep. Accuracy = 0.4888888888888889
Subject 14, Sub-Indep. Accuracy = 0.5333333333333333
Subject 15, Sub-Indep. Accuracy = 0.5055555555555555
Subject 16, Sub-Indep. Accuracy = 0.5888888888888889
Subject 17, Sub-Indep. Accuracy = 0.5055555555555555
Subject 18, Sub-Indep. Accuracy = 0.8333333333333334
Subject 19, Sub-Indep. Accuracy = 0.7111111111111111
Subject 20, Sub

Subject 53, Sub-Indep. Accuracy = 0.5
Subject 54, Sub-Indep. Accuracy = 0.5444444444444444
Average: 0.6316872427983539

K = 9

Subject 1, Sub-Indep. Accuracy = 0.6666666666666666
Subject 2, Sub-Indep. Accuracy = 0.5
Subject 3, Sub-Indep. Accuracy = 0.8666666666666667
Subject 4, Sub-Indep. Accuracy = 0.5388888888888889
Subject 5, Sub-Indep. Accuracy = 0.7277777777777777
Subject 6, Sub-Indep. Accuracy = 0.6888888888888889
Subject 7, Sub-Indep. Accuracy = 0.48333333333333334
Subject 8, Sub-Indep. Accuracy = 0.5833333333333334
Subject 9, Sub-Indep. Accuracy = 0.7888888888888889
Subject 10, Sub-Indep. Accuracy = 0.5222222222222223
Subject 11, Sub-Indep. Accuracy = 0.49444444444444446
Subject 12, Sub-Indep. Accuracy = 0.5222222222222223
Subject 13, Sub-Indep. Accuracy = 0.5777777777777777
Subject 14, Sub-Indep. Accuracy = 0.5666666666666667
Subject 15, Sub-Indep. Accuracy = 0.49444444444444446
Subject 16, Sub-Indep. Accuracy = 0.5944444444444444
Subject 17, Sub-Indep. Accuracy = 0.8055555555

Subject 50, Sub-Indep. Accuracy = 0.5555555555555556
Subject 51, Sub-Indep. Accuracy = 0.5944444444444444
Subject 52, Sub-Indep. Accuracy = 0.75
Subject 53, Sub-Indep. Accuracy = 0.5666666666666667
Subject 54, Sub-Indep. Accuracy = 0.5833333333333334
Average: 0.6210905349794239

K = 7

Subject 1, Sub-Indep. Accuracy = 0.6166666666666667
Subject 2, Sub-Indep. Accuracy = 0.5
Subject 3, Sub-Indep. Accuracy = 0.7166666666666667
Subject 4, Sub-Indep. Accuracy = 0.5444444444444444
Subject 5, Sub-Indep. Accuracy = 0.8166666666666667
Subject 6, Sub-Indep. Accuracy = 0.7666666666666667
Subject 7, Sub-Indep. Accuracy = 0.4388888888888889
Subject 8, Sub-Indep. Accuracy = 0.5888888888888889
Subject 9, Sub-Indep. Accuracy = 0.75
Subject 10, Sub-Indep. Accuracy = 0.5222222222222223
Subject 11, Sub-Indep. Accuracy = 0.5055555555555555
Subject 12, Sub-Indep. Accuracy = 0.55
Subject 13, Sub-Indep. Accuracy = 0.5222222222222223
Subject 14, Sub-Indep. Accuracy = 0.5944444444444444
Subject 15, Sub-Indep. 

Subject 48, Sub-Indep. Accuracy = 0.5222222222222223
Subject 49, Sub-Indep. Accuracy = 0.5388888888888889
Subject 50, Sub-Indep. Accuracy = 0.5055555555555555
Subject 51, Sub-Indep. Accuracy = 0.6222222222222222
Subject 52, Sub-Indep. Accuracy = 0.6666666666666666
Subject 53, Sub-Indep. Accuracy = 0.5722222222222222
Subject 54, Sub-Indep. Accuracy = 0.5722222222222222
Average: 0.6029835390946503

K = 5

Subject 1, Sub-Indep. Accuracy = 0.6444444444444445
Subject 2, Sub-Indep. Accuracy = 0.5
Subject 3, Sub-Indep. Accuracy = 0.8611111111111112
Subject 4, Sub-Indep. Accuracy = 0.5611111111111111
Subject 5, Sub-Indep. Accuracy = 0.7888888888888889
Subject 6, Sub-Indep. Accuracy = 0.6666666666666666
Subject 7, Sub-Indep. Accuracy = 0.45555555555555555
Subject 8, Sub-Indep. Accuracy = 0.5166666666666667
Subject 9, Sub-Indep. Accuracy = 0.6277777777777778
Subject 10, Sub-Indep. Accuracy = 0.6055555555555555
Subject 11, Sub-Indep. Accuracy = 0.5
Subject 12, Sub-Indep. Accuracy = 0.544444444444

Subject 44, Sub-Indep. Accuracy = 0.7777777777777778
Subject 45, Sub-Indep. Accuracy = 0.6055555555555555
Subject 46, Sub-Indep. Accuracy = 0.5722222222222222
Subject 47, Sub-Indep. Accuracy = 0.4722222222222222
Subject 48, Sub-Indep. Accuracy = 0.4444444444444444
Subject 49, Sub-Indep. Accuracy = 0.6111111111111112
Subject 50, Sub-Indep. Accuracy = 0.5222222222222223
Subject 51, Sub-Indep. Accuracy = 0.5388888888888889
Subject 52, Sub-Indep. Accuracy = 0.7111111111111111
Subject 53, Sub-Indep. Accuracy = 0.49444444444444446
Subject 54, Sub-Indep. Accuracy = 0.5277777777777778
Average: 0.6112139917695474

N = 5

K = 3

Subject 1, Sub-Indep. Accuracy = 0.7333333333333333
Subject 2, Sub-Indep. Accuracy = 0.5888888888888889
Subject 3, Sub-Indep. Accuracy = 0.7777777777777778
Subject 4, Sub-Indep. Accuracy = 0.6222222222222222
Subject 5, Sub-Indep. Accuracy = 0.5833333333333334
Subject 6, Sub-Indep. Accuracy = 0.5722222222222222
Subject 7, Sub-Indep. Accuracy = 0.45555555555555555
Subject 

Subject 40, Sub-Indep. Accuracy = 0.5722222222222222
Subject 41, Sub-Indep. Accuracy = 0.5111111111111111
Subject 42, Sub-Indep. Accuracy = 0.5444444444444444
Subject 43, Sub-Indep. Accuracy = 0.6333333333333333
Subject 44, Sub-Indep. Accuracy = 0.9444444444444444
Subject 45, Sub-Indep. Accuracy = 0.6777777777777778
Subject 46, Sub-Indep. Accuracy = 0.5111111111111111
Subject 47, Sub-Indep. Accuracy = 0.5277777777777778
Subject 48, Sub-Indep. Accuracy = 0.4388888888888889
Subject 49, Sub-Indep. Accuracy = 0.6111111111111112
Subject 50, Sub-Indep. Accuracy = 0.5166666666666667
Subject 51, Sub-Indep. Accuracy = 0.6055555555555555
Subject 52, Sub-Indep. Accuracy = 0.6777777777777778
Subject 53, Sub-Indep. Accuracy = 0.5277777777777778
Subject 54, Sub-Indep. Accuracy = 0.4888888888888889
Average: 0.6303497942386832

K = 9

Subject 1, Sub-Indep. Accuracy = 0.6777777777777778
Subject 2, Sub-Indep. Accuracy = 0.5055555555555555
Subject 3, Sub-Indep. Accuracy = 0.8833333333333333
Subject 4, Su

In [19]:
metric = "LEM"
sess = "sess01"

for n in range(2,6):
    print("\nN = " + str(n))
    for k in range(3, 10, 2):
        print("\nK = " + str(k) + "\n")
        res = csp.mainloop(sess, metric, n=n, samples=10, k=k)


N = 2

K = 3

Subject 1, Sub-Indep. Accuracy = 0.6722222222222223
Subject 2, Sub-Indep. Accuracy = 0.5444444444444444
Subject 3, Sub-Indep. Accuracy = 0.55
Subject 4, Sub-Indep. Accuracy = 0.5333333333333333
Subject 5, Sub-Indep. Accuracy = 0.7166666666666667
Subject 6, Sub-Indep. Accuracy = 0.6444444444444445
Subject 7, Sub-Indep. Accuracy = 0.46111111111111114
Subject 8, Sub-Indep. Accuracy = 0.5222222222222223
Subject 9, Sub-Indep. Accuracy = 0.6444444444444445
Subject 10, Sub-Indep. Accuracy = 0.55
Subject 11, Sub-Indep. Accuracy = 0.49444444444444446
Subject 12, Sub-Indep. Accuracy = 0.5833333333333334
Subject 13, Sub-Indep. Accuracy = 0.5111111111111111
Subject 14, Sub-Indep. Accuracy = 0.5
Subject 15, Sub-Indep. Accuracy = 0.5111111111111111
Subject 16, Sub-Indep. Accuracy = 0.6555555555555556
Subject 17, Sub-Indep. Accuracy = 0.6944444444444444
Subject 18, Sub-Indep. Accuracy = 0.8277777777777777
Subject 19, Sub-Indep. Accuracy = 0.7277777777777777
Subject 20, Sub-Indep. Accur

Subject 54, Sub-Indep. Accuracy = 0.49444444444444446
Average: 0.640946502057613

K = 9

Subject 1, Sub-Indep. Accuracy = 0.6222222222222222
Subject 2, Sub-Indep. Accuracy = 0.5
Subject 3, Sub-Indep. Accuracy = 0.8777777777777778
Subject 4, Sub-Indep. Accuracy = 0.5222222222222223
Subject 5, Sub-Indep. Accuracy = 0.8166666666666667
Subject 6, Sub-Indep. Accuracy = 0.6722222222222223
Subject 7, Sub-Indep. Accuracy = 0.45555555555555555
Subject 8, Sub-Indep. Accuracy = 0.5166666666666667
Subject 9, Sub-Indep. Accuracy = 0.8111111111111111
Subject 10, Sub-Indep. Accuracy = 0.55
Subject 11, Sub-Indep. Accuracy = 0.4722222222222222
Subject 12, Sub-Indep. Accuracy = 0.5944444444444444
Subject 13, Sub-Indep. Accuracy = 0.5222222222222223
Subject 14, Sub-Indep. Accuracy = 0.5722222222222222
Subject 15, Sub-Indep. Accuracy = 0.49444444444444446
Subject 16, Sub-Indep. Accuracy = 0.5722222222222222
Subject 17, Sub-Indep. Accuracy = 0.7388888888888889
Subject 18, Sub-Indep. Accuracy = 0.75
Subject

Subject 50, Sub-Indep. Accuracy = 0.5722222222222222
Subject 51, Sub-Indep. Accuracy = 0.6
Subject 52, Sub-Indep. Accuracy = 0.7055555555555556
Subject 53, Sub-Indep. Accuracy = 0.5555555555555556
Subject 54, Sub-Indep. Accuracy = 0.5166666666666667
Average: 0.6182098765432097

K = 7

Subject 1, Sub-Indep. Accuracy = 0.6388888888888888
Subject 2, Sub-Indep. Accuracy = 0.7611111111111111
Subject 3, Sub-Indep. Accuracy = 0.6666666666666666
Subject 4, Sub-Indep. Accuracy = 0.5388888888888889
Subject 5, Sub-Indep. Accuracy = 0.8166666666666667
Subject 6, Sub-Indep. Accuracy = 0.7722222222222223
Subject 7, Sub-Indep. Accuracy = 0.46111111111111114
Subject 8, Sub-Indep. Accuracy = 0.5
Subject 9, Sub-Indep. Accuracy = 0.6888888888888889
Subject 10, Sub-Indep. Accuracy = 0.5777777777777777
Subject 11, Sub-Indep. Accuracy = 0.4722222222222222
Subject 12, Sub-Indep. Accuracy = 0.6
Subject 13, Sub-Indep. Accuracy = 0.5055555555555555
Subject 14, Sub-Indep. Accuracy = 0.5722222222222222
Subject 15

Subject 48, Sub-Indep. Accuracy = 0.46111111111111114
Subject 49, Sub-Indep. Accuracy = 0.4888888888888889
Subject 50, Sub-Indep. Accuracy = 0.5277777777777778
Subject 51, Sub-Indep. Accuracy = 0.6055555555555555
Subject 52, Sub-Indep. Accuracy = 0.6722222222222223
Subject 53, Sub-Indep. Accuracy = 0.5666666666666667
Subject 54, Sub-Indep. Accuracy = 0.5444444444444444
Average: 0.5972222222222222

K = 5

Subject 1, Sub-Indep. Accuracy = 0.6
Subject 2, Sub-Indep. Accuracy = 0.5444444444444444
Subject 3, Sub-Indep. Accuracy = 0.7944444444444444
Subject 4, Sub-Indep. Accuracy = 0.6
Subject 5, Sub-Indep. Accuracy = 0.6722222222222223
Subject 6, Sub-Indep. Accuracy = 0.7277777777777777
Subject 7, Sub-Indep. Accuracy = 0.45
Subject 8, Sub-Indep. Accuracy = 0.5333333333333333
Subject 9, Sub-Indep. Accuracy = 0.7166666666666667
Subject 10, Sub-Indep. Accuracy = 0.5833333333333334
Subject 11, Sub-Indep. Accuracy = 0.4777777777777778
Subject 12, Sub-Indep. Accuracy = 0.5166666666666667
Subject 1

Subject 43, Sub-Indep. Accuracy = 0.6388888888888888
Subject 44, Sub-Indep. Accuracy = 0.8166666666666667
Subject 45, Sub-Indep. Accuracy = 0.5944444444444444
Subject 46, Sub-Indep. Accuracy = 0.5555555555555556
Subject 47, Sub-Indep. Accuracy = 0.5611111111111111
Subject 48, Sub-Indep. Accuracy = 0.4444444444444444
Subject 49, Sub-Indep. Accuracy = 0.6111111111111112
Subject 50, Sub-Indep. Accuracy = 0.5388888888888889
Subject 51, Sub-Indep. Accuracy = 0.5666666666666667
Subject 52, Sub-Indep. Accuracy = 0.7222222222222222
Subject 53, Sub-Indep. Accuracy = 0.5333333333333333
Subject 54, Sub-Indep. Accuracy = 0.5166666666666667
Average: 0.6246913580246912

N = 5

K = 3

Subject 1, Sub-Indep. Accuracy = 0.7111111111111111
Subject 2, Sub-Indep. Accuracy = 0.7055555555555556
Subject 3, Sub-Indep. Accuracy = 0.5277777777777778
Subject 4, Sub-Indep. Accuracy = 0.6166666666666667
Subject 5, Sub-Indep. Accuracy = 0.5333333333333333
Subject 6, Sub-Indep. Accuracy = 0.5777777777777777
Subject 7

Subject 41, Sub-Indep. Accuracy = 0.5333333333333333
Subject 42, Sub-Indep. Accuracy = 0.5444444444444444
Subject 43, Sub-Indep. Accuracy = 0.6333333333333333
Subject 44, Sub-Indep. Accuracy = 0.9055555555555556
Subject 45, Sub-Indep. Accuracy = 0.6
Subject 46, Sub-Indep. Accuracy = 0.5166666666666667
Subject 47, Sub-Indep. Accuracy = 0.5055555555555555
Subject 48, Sub-Indep. Accuracy = 0.49444444444444446
Subject 49, Sub-Indep. Accuracy = 0.6166666666666667
Subject 50, Sub-Indep. Accuracy = 0.49444444444444446
Subject 51, Sub-Indep. Accuracy = 0.5722222222222222
Subject 52, Sub-Indep. Accuracy = 0.75
Subject 53, Sub-Indep. Accuracy = 0.5555555555555556
Subject 54, Sub-Indep. Accuracy = 0.5055555555555555
Average: 0.6329218106995884

K = 9

Subject 1, Sub-Indep. Accuracy = 0.6444444444444445
Subject 2, Sub-Indep. Accuracy = 0.5
Subject 3, Sub-Indep. Accuracy = 0.8555555555555555
Subject 4, Sub-Indep. Accuracy = 0.5333333333333333
Subject 5, Sub-Indep. Accuracy = 0.8111111111111111
Subj